In [2]:
# Standard library imports
import os
import requests

# Third party imports
import ee
import ee.mapclient
import folium
from google.oauth2 import service_account
from osgeo import gdal
from pyproj import Transformer

from dotenv import load_dotenv

#Specify the path to the service account key file (found in .env)
load_dotenv()
key_path = os.getenv('GEE_AUTH_KEY_PATH')

# Define the correct scope for Earth Engine
SCOPES = ['https://www.googleapis.com/auth/earthengine']

# Load the service account credentials
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=SCOPES
)

# Initialize Earth Engine
ee.Initialize(credentials=credentials)
print("Earth Engine initialized successfully!")

Earth Engine initialized successfully!


In [ ]:
# Define the bounding box in epsg 6514
upper_left = [252500, 287500]
bottom_right = [256000, 278000]

buffer_amt = 1e4

upper_left_buffered = [upper_left[0] - buffer_amt, upper_left[1] + buffer_amt]
bottom_right_buffered = [bottom_right[0] + buffer_amt, bottom_right[1] - buffer_amt]

# Create a transformer object to convert from EPSG:6514 to EPSG:4326
transformer = Transformer.from_crs("EPSG:6514", "EPSG:4326", always_xy=True)

# Convert the coordinates
upper_left_latlon = transformer.transform(upper_left_buffered[0], upper_left_buffered[1])
bottom_right_latlon = transformer.transform(bottom_right_buffered[0], bottom_right_buffered[1])

# Print the results
print("Upper Left (LatLon):", upper_left_latlon)
print("Bottom Right (LatLon):", bottom_right_latlon)

# Define the bounding box in lat/lon (EPSG:4326)
bounding_box = ee.Geometry.Polygon([
    [upper_left_latlon,
     [bottom_right_latlon[0], upper_left_latlon[1]],  # Upper-right corner
     bottom_right_latlon,
     [upper_left_latlon[0], bottom_right_latlon[1]],  # Lower-left corner
     upper_left_latlon]  # Closing the polygon
], proj='EPSG:4326')

# Load the National Elevation Dataset (NED)
ned = ee.Image("USGS/3DEP/10m")

# Clip the NED to the bounding box
ned_clipped = ned.clip(bounding_box)

In [10]:

# Define visualization parameters
vis_params = {'min': 750, 'max': 3000, 'palette': ['440154', '482878', '3E4A89', '31688E', '26828E', '1F9E89', '35B779', '6DCD59', 'B4DE2C', 'FDE725']}

# Create a folium map object.
centroid = bounding_box.centroid()
centroid_coords = centroid.coordinates().getInfo()
m = folium.Map(location=[centroid_coords[1], centroid_coords[0]], zoom_start=12)

# Add the NED layer to the map
map_id_dict = ned_clipped.getMapId(vis_params)
folium.TileLayer(
    tiles=map_id_dict['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='NED',
).add_to(m)


# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map
display(m)

In [11]:
output_dir = 'data/raster/ned'
os.makedirs(output_dir, exist_ok=True)

# Set the GTIFF_SRS_SOURCE configuration option to EPSG
gdal.SetConfigOption('GTIFF_SRS_SOURCE', 'EPSG')

# Choose a value that doesn't exist in your data and doesn't have meaning in the context of your data.
remapped_image = ned_clipped.unmask(9999)

# Set an explicit "no data" value to null in the download URL parameters
url = remapped_image.getDownloadURL({
    'scale': 30,
    'crs': 'EPSG:6514',
    'region': bounding_box,
    'format': 'GeoTIFF',
    'formatOptions': {
        'cloudOptimized': True,
        'noData': 9999
    }
})

response = requests.get(url)

# Create output file path
output_file = os.path.join(output_dir, f"ned_temp.tif")

# Write response content directly to final file
with open(output_file, 'wb') as f:
    f.write(response.content)

print(f"Downloaded file {output_file}")

# Set nodata value to 255
dataset = gdal.Open(output_file, gdal.GA_Update)
band = dataset.GetRasterBand(1)
band.SetNoDataValue(255)
dataset = None

Downloaded file data/raster/ned/ned_temp.tif


/opt/miniconda3/envs/calibrate-rap/lib/python3.10/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
Warning 1: TIFFWriteDirectorySec:Creating TIFF with legacy Deflate codec identifier, COMPRESSION_ADOBE_DEFLATE is more widely supported


In [12]:
input_file = os.path.join(output_dir, f"ned_temp.tif")
output_file = input_file.replace("_temp", "")
# Use gdal_translate to crop the image in place
gdal.Translate(output_file, input_file, projWin=[upper_left[0], upper_left[1], bottom_right[0], bottom_right[1]])

# Remove original and rename temp file
os.remove(input_file)

print(f"Cropped file {input_file}")

Cropped file data/raster/ned/ned_temp.tif
